# Setup setting and seed

In [1]:
from multilevel_sabre import MultiLevelSabre
from pathlib import Path
from typing import Dict, List, Iterable
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes import SabreLayout
from qiskit.transpiler import CouplingMap, PassManager
from IPython.display import display
import json
import time


seed = 42

# Setup circuits

In [2]:
def gather_qasm_files(paths: Iterable[str]) -> List[Path]:
    """Expand a list of files/dirs into a sorted list of .qasm files."""
    result: List[Path] = []
    for p in paths:
        path = Path(p)
        if path.is_dir():
            result.extend(sorted(path.glob("*.qasm")))
        elif path.is_file() and path.suffix.lower() == ".qasm":
            result.append(path)
        else:
            print(f"Skipping {p}: not a .qasm file or directory.")
    return sorted(set(result))

filepath = "circuits_iso/"
qasm_files = gather_qasm_files([filepath])
print(f"Found {len(qasm_files)} .qasm files in {filepath}")

Found 7 .qasm files in circuits_iso/


In [3]:
circuits_dict = {}
for q in qasm_files:
    circuit = QuantumCircuit.from_qasm_file(q)
    print(f"Visualizing {q.name} with {circuit.num_qubits} qubits and {circuit.size()} gates")
    #display(circuit.draw('mpl', idle_wires=False, fold=-1))
    circuits_dict[q.name] = circuit


Visualizing cat_n65.qasm with 65 qubits and 130 gates
Visualizing dnn_n51.qasm with 51 qubits and 274 gates
Visualizing ghz_n78.qasm with 78 qubits and 156 gates
Visualizing ising_n98.qasm with 98 qubits and 1170 gates
Visualizing knn_67.qasm with 67 qubits and 102 gates
Visualizing multiplier_n45.qasm with 45 qubits and 698 gates
Visualizing swap_test_n83.qasm with 83 qubits and 126 gates


In [4]:
circuits_dict

{'cat_n65.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x116aa8550>,
 'dnn_n51.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x111a12d50>,
 'ghz_n78.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x116a7f4d0>,
 'ising_n98.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1202a8e50>,
 'knn_67.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1202b8a50>,
 'multiplier_n45.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1160d1d10>,
 'swap_test_n83.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x116ab92d0>}

In [5]:
qc_name = "cat_n65.qasm"

# Setup coupling map

In [6]:
from qiskit_ibm_runtime.fake_provider.backends.sherbrooke import FakeSherbrooke
fake_backend = FakeSherbrooke()
coupling_map = fake_backend.coupling_map   
coupling_map.make_symmetric()
print(f"Number of qubits: {coupling_map.size()}")
# coupling_map.draw()


Number of qubits: 127


# Setup Passmangers

In [7]:
from qiskit.transpiler.passes.layout import VF2Layout
from qiskit.transpiler.passes.basis  import Unroll3qOrMore
number_of_trial = 2500


vf2_layout = VF2Layout(
    coupling_map = coupling_map, 
    seed = seed, 
)
pm_sabre = PassManager([
    Unroll3qOrMore(),
    vf2_layout,
    SabreLayout(
        coupling_map = coupling_map, 
        seed = seed, 
        layout_trials=number_of_trial, 
        skip_routing=False)
])

print(f"Stop reason: {vf2_layout.property_set['VF2Layout_stop_reason']}")

Stop reason: None


# Transpile circuit

In [8]:
def count_swaps(circuit):
    return sum(1 for instruction in circuit.data if instruction.operation.name == 'swap')

In [9]:

# Sabre pass manager
qc_tr_sabre = pm_sabre.run(circuits_dict[qc_name])
print(f"Number of SWAPs (sabre): {count_swaps(qc_tr_sabre)}")
print(f"2Q Depth (sabre): {qc_tr_sabre.depth(lambda x: x.operation.num_qubits == 2)}\n")
print(f"Stop reason: {vf2_layout.property_set['VF2Layout_stop_reason']}")


Number of SWAPs (sabre): 14
2Q Depth (sabre): 78

Stop reason: VF2LayoutStopReason.SOLUTION_FOUND


In [10]:
# transpile all of the circuits
for qc_name, qc in circuits_dict.items():
    print(f"Transpiling {qc_name} with {qc.num_qubits} qubits and {qc.size()} gates")
    
    qc_tr_sabre = pm_sabre.run(circuits_dict[qc_name])
    print(f"Number of SWAPs (sabre): {count_swaps(qc_tr_sabre)}")
    print(f"2Q Depth (sabre): {qc_tr_sabre.depth(lambda x: x.operation.num_qubits == 2)}")
    print(f"Stop reason: {vf2_layout.property_set['VF2Layout_stop_reason']}\n")

Transpiling cat_n65.qasm with 65 qubits and 130 gates
Number of SWAPs (sabre): 14
2Q Depth (sabre): 78
Stop reason: VF2LayoutStopReason.SOLUTION_FOUND

Transpiling dnn_n51.qasm with 51 qubits and 274 gates
Number of SWAPs (sabre): 131
2Q Depth (sabre): 208
Stop reason: VF2LayoutStopReason.NO_SOLUTION_FOUND

Transpiling ghz_n78.qasm with 78 qubits and 156 gates
Number of SWAPs (sabre): 16
2Q Depth (sabre): 93
Stop reason: VF2LayoutStopReason.SOLUTION_FOUND

Transpiling ising_n98.qasm with 98 qubits and 1170 gates
Number of SWAPs (sabre): 17
2Q Depth (sabre): 10
Stop reason: VF2LayoutStopReason.SOLUTION_FOUND

Transpiling knn_67.qasm with 67 qubits and 102 gates
Number of SWAPs (sabre): 106
2Q Depth (sabre): 262
Stop reason: VF2LayoutStopReason.NO_SOLUTION_FOUND

Transpiling multiplier_n45.qasm with 45 qubits and 698 gates
Number of SWAPs (sabre): 1613
2Q Depth (sabre): 2364
Stop reason: VF2LayoutStopReason.NO_SOLUTION_FOUND

Transpiling swap_test_n83.qasm with 83 qubits and 126 gates
Nu